In [1]:
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -5, 10
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)
bounds = [(lowerBound,upperBound) for i in range(N)]

def f(x):
    #Sphere
    # x in interval [-30, -30]
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock
    #za epsilon = 10e-30 dao nulu
    # x in interval [-30, -30]
    #Sum = 0
    #for i in range(0, N-1):
    #    Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    # x in interval [-5, -10]
    Sum1, Sum2 = 0, 0
    for i in range(0,N):
        Sum1 = Sum1 + (x[i])**2
        Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(N)]
u = [upperBound for i in range(N)]

In [2]:
import copy
import math
import numpy as np


'''
    Pure Python/Numpy implementation of the Nelder-Mead algorithm.
    Reference: https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method
'''


def SimplexLocalSearch(x_start, no_improve_thr,step, max_iter,
                no_improv_break=10,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5):
    '''
        @param f (function): function to optimize, must return a scalar score
            and operate over a numpy array of the same dimensions as x_start
        @param x_start (numpy array): initial position
        @param step (float): look-around radius in initial step
        @no_improv_thr,  no_improv_break (float, int): break after no_improv_break iterations with
            an improvement lower than no_improv_thr
        @max_iter (int): always break after this number of iterations.
            Set it to 0 to loop indefinitely.
        @alpha, gamma, rho, sigma (floats): parameters of the algorithm
            (see Wikipedia page for reference)
        return: tuple (best parameter array, best score)
    '''

    # init
    dim = len(x_start)
    prev_best = f(x_start)
    no_improv = 0
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = copy.copy(x_start)
        x[i] = x[i] + step
        score = f(x)
        res.append([x, score])

    # simplex iter
    iters = 0
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # break after max_iter
        if max_iter and iters >= max_iter:
            return res[0][0],iters,res
        iters += 1

        # break after no_improv_break iterations with no improvement
        #print('...best so far:', best)

        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0][0],iters,res

        # centroid
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)

        # reflection
        #print("reflection",iters)
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr)
        if res[0][1] <= rscore < res[-2][1]:
            del res[-1]
            res.append([xr, rscore])
            continue

        # expansion
        #print("expansion",iters)
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue

        # contraction
        #print("contraction",iters)
        xc = x0 + rho*(x0 - res[-1][0])
        cscore = f(xc)
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            continue

        # reduction
        #print("reduction",iters)
        x1 = res[0][0]
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx)
            nres.append([redx, score])
        res = nres

In [3]:
def Delta(x):
    #import pdb
    #set_trace()
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [4]:
from scipy.optimize import minimize

def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

def restartSimplex(x,Lambda):
    N = len(x)
    sim = np.empty([N+1,N])
    sim[0] = x
    for j in range(1,N+1):
        sim[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N)
    

    for i in range(len(sim)):
     
        # Find the minimum element in remaining
        # unsorted array
        min_idx = i
        for j in range(i+1, len(sim)):
            if f(sim[min_idx]) > f(sim[j]):
                min_idx = j

        # Swap the found minimum element with
        # the first element       
        temp = copy.copy(sim[i])
        sim[i] = copy.copy(sim[min_idx])
        sim[min_idx] = copy.copy(temp)
        
    return sim

In [11]:
Nfeval = 1

def callbackFsmall(Xi):
    global Nfeval
    print('small: {0:4d}   {1: 3.6f}'.format(Nfeval, f(Xi)))
    Nfeval += 1

def callbackFbig(Xi):
    global Nfeval
    print('big: {0:4d}   {1: 3.6f}'.format(Nfeval, f(Xi)))
    Nfeval += 1

In [14]:
def DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M, Gamma = 100,Beta=-0.5):
    k = 0
    it = 0
    x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
    
    sim = restartSimplex(x,Lambda)
    #set_trace()
    while(k < M):
        #set_trace()
        temp = minimize(f, x, method='nelder-mead',bounds=bounds, tol=None, callback=callbackFsmall, 
                        options={'maxiter': M-k, 'maxfev': None, 'disp': True,
                                 'return_all': False, 'initial_simplex': sim, 'xatol': False, 
                                 'fatol': Epsilon, 'adaptive': False})
        x = copy.copy(temp['x'])
        _k = temp['nfev']
        s = temp['final_simplex']
        sim = s[0]
        values = s[1]
        #print(values)
        #set_trace()
        #for i in range(0,N+1):print(f(sim[i]))
        #set_trace()
        
        
        #import pdb
        #pdb.set_trace()
        k = k + _k
        try: xBest
        except NameError: xBest = None
        
        if xBest is None or f(x) < f(xBest):
            temp = minimize(f, x, method='nelder-mead',bounds=bounds, tol=None, callback=callbackFbig, 
                        options={'maxiter': M-k, 'maxfev': None, 'disp': False, 
                                 'return_all': True, 'initial_simplex': sim, 'xatol': False, 
                                 'fatol': EpsilonApostrophe, 'adaptive': False})
            xBest = copy.copy(temp['x'])
            _k = temp['nfev']
            #s = temp['final_simplex']
            #sim = s[0]
            k = k + _k
            print("***NEW BEST VALUE*** iteration: ", k, "| best found value: ", f(xBest))
            
            
        if xBest is not None and k//10000 > it:
            it = k//10000
            print("iteration:", it * 10000, "| best found value: ", f(xBest))
            
        #set_trace()
        #for i in range(0,N+1):print(f(sim[i]))
        #set_trace()
        
            
        
        values = s[1]
            
        #for i in range(0,N):
        #    sim[i][i] += 10e-5
        
        #same = True
        #for i in range(1,N+1):
        #    if(values[0] - values[i] > 0):
        #        same = False
        #if(same):
            #set_trace()
        #    sim = restartSimplex(sim[0],Lambda)
        
        #for i in range(0,N+1):
        #    values[i] = f(sim[i])
        
        while f(x) < f(sim[-1]):
            #set_trace()
            
            #set_trace()
            #for i in range(0,N+1):print(f(sim[i]))
            #set_trace()
            
            
            sim[-1] = copy.copy(x)
            values[-1] = f(x)
            #set_trace()

            
            for i in range(N,0, -1):
                if(values[i] < values[i-1]):
                    temp1, temp2 = copy.copy(sim[i]), copy.copy(values[i])
                    sim[i], values[i] = copy.copy(sim[i-1]), copy.copy(values[i-1])
                    sim[i-1], values[i-1] = copy.copy(temp1), copy.copy(temp2)
                    



            x0 = np.zeros(N)

            for i in range(1,N+1):
                x0 = x0 + sim[i]
            x0 = x0 / N

            x = copy.copy(Gamma*sim[0] + (1-Gamma) * x0)

            sim[0] = copy.copy(x)
            values[0] = f(x)

            for i in range(0,N):
                if(values[i] > values[i+1]):
                    temp1, temp2 = copy.copy(sim[i]), copy.copy(values[i])
                    sim[i], values[i] = copy.copy(sim[i+1]), copy.copy(values[i+1])
                    sim[i+1], values[i+1] = copy.copy(temp1), copy.copy(temp2)
                

        #print(Delta(x))
        
        
        
    return xBest

In [ ]:
Lambda = 10
M = 1000000
Epsilon = 10
#set_trace()
EpsilonApostrophe = 0
print(DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M))

small: 11453    14.371882
small: 11454    14.371882
small: 11455    14.371882
small: 11456    14.371882
small: 11457    14.371882
small: 11458    14.371882
small: 11459    14.371882
small: 11460    14.107494
small: 11461    14.107494
small: 11462    14.107494
small: 11463    14.107494
small: 11464    14.107494
small: 11465    14.107494
small: 11466    14.107494
small: 11467    13.740816
small: 11468    13.546291
small: 11469    13.546291
small: 11470    13.546291
small: 11471    13.247354
small: 11472    13.247354
small: 11473    13.247354
small: 11474    12.659502
small: 11475    12.659502
small: 11476    12.659502
small: 11477    12.659502
small: 11478    12.659502
small: 11479    12.659502
small: 11480    12.659502
small: 11481    12.659502
small: 11482    12.659502
small: 11483    12.659502
small: 11484    12.659502
small: 11485    12.659502
small: 11486    12.659502
small: 11487    12.521473
small: 11488    12.521473
small: 11489    12.521473
small: 11490    12.521473
small: 11491

small: 11835    10.976924
small: 11836    10.976924
small: 11837    10.976924
small: 11838    10.976924
small: 11839    10.976924
small: 11840    10.976924
small: 11841    10.976924
small: 11842    10.976924
small: 11843    10.976924
small: 11844    10.976924
small: 11845    10.976897
small: 11846    10.976609
small: 11847    10.976609
small: 11848    10.976609
small: 11849    10.976609
small: 11850    10.976609
small: 11851    10.976609
small: 11852    10.976480
small: 11853    10.976426
small: 11854    10.976426
small: 11855    10.976426
small: 11856    10.976426
small: 11857    10.976426
small: 11858    10.976426
small: 11859    10.976390
small: 11860    10.976197
small: 11861    10.976197
small: 11862    10.976197
small: 11863    10.976197
small: 11864    10.976197
small: 11865    10.976197
small: 11866    10.976084
small: 11867    10.976084
small: 11868    10.976084
small: 11869    10.976084
small: 11870    10.976084
small: 11871    10.976084
small: 11872    10.976084
small: 11873

small: 12238    10.975617
small: 12239    10.975617
small: 12240    10.975617
small: 12241    10.975617
small: 12242    10.975617
small: 12243    10.975617
small: 12244    10.975617
small: 12245    10.975617
small: 12246    10.975617
small: 12247    10.975617
small: 12248    10.975617
small: 12249    10.975617
small: 12250    10.975617
small: 12251    10.975617
small: 12252    10.975617
small: 12253    10.975617
small: 12254    10.975617
small: 12255    10.975617
small: 12256    10.975617
small: 12257    10.975617
small: 12258    10.975617
small: 12259    10.975617
small: 12260    10.975617
small: 12261    10.975617
small: 12262    10.975617
small: 12263    10.975617
small: 12264    10.975617
small: 12265    10.975617
small: 12266    10.975617
small: 12267    10.975617
small: 12268    10.975617
small: 12269    10.975617
small: 12270    10.975617
small: 12271    10.975617
small: 12272    10.975617
small: 12273    10.975617
small: 12274    10.975617
small: 12275    10.975617
small: 12276

small: 12604    10.975617
small: 12605    10.975617
small: 12606    10.975617
small: 12607    10.975617
small: 12608    10.975617
small: 12609    10.975617
small: 12610    10.975617
small: 12611    10.975617
small: 12612    10.975617
small: 12613    10.975617
small: 12614    10.975617
small: 12615    10.975617
small: 12616    10.975617
small: 12617    10.975617
small: 12618    10.975617
small: 12619    10.975617
small: 12620    10.975617
small: 12621    10.975617
small: 12622    10.975617
small: 12623    10.975617
small: 12624    10.975617
small: 12625    10.975617
small: 12626    10.975617
small: 12627    10.975617
small: 12628    10.975617
small: 12629    10.975617
small: 12630    10.975617
small: 12631    10.975617
small: 12632    10.975617
small: 12633    10.975617
small: 12634    10.975617
small: 12635    10.975617
small: 12636    10.975617
small: 12637    10.975617
small: 12638    10.975617
small: 12639    10.975617
small: 12640    10.975617
small: 12641    10.975617
small: 12642

small: 12940    10.975617
small: 12941    10.975617
small: 12942    10.975617
small: 12943    10.975617
small: 12944    10.975617
small: 12945    10.975617
small: 12946    10.975617
small: 12947    10.975617
small: 12948    10.975617
small: 12949    10.975617
small: 12950    10.975617
small: 12951    10.975617
small: 12952    10.975617
small: 12953    10.975617
small: 12954    10.975617
small: 12955    10.975617
small: 12956    10.975617
small: 12957    10.975617
small: 12958    10.975617
small: 12959    10.975617
small: 12960    10.975617
small: 12961    10.975617
small: 12962    10.975617
small: 12963    10.975617
small: 12964    10.975617
small: 12965    10.975617
small: 12966    10.975617
small: 12967    10.975617
small: 12968    10.975617
small: 12969    10.975617
small: 12970    10.975617
small: 12971    10.975617
small: 12972    10.975617
small: 12973    10.975617
small: 12974    10.975617
small: 12975    10.975617
small: 12976    10.975617
small: 12977    10.975617
small: 12978

...

small: 118738    10.975617
small: 118739    10.975617
small: 118740    10.975617
small: 118741    10.975617
small: 118742    10.975617
small: 118743    10.975617
small: 118744    10.975617
small: 118745    10.975617
small: 118746    10.975617
small: 118747    10.975617
small: 118748    10.975617
small: 118749    10.975617
small: 118750    10.975617
small: 118751    10.975617
small: 118752    10.975617
small: 118753    10.975617
small: 118754    10.975617
small: 118755    10.975617
small: 118756    10.975617
small: 118757    10.975617
small: 118758    10.975617
small: 118759    10.975617
small: 118760    10.975617
small: 118761    10.975617
small: 118762    10.975617
small: 118763    10.975617
small: 118764    10.975617
small: 118765    10.975617
small: 118766    10.975617
small: 118767    10.975617
small: 118768    10.975617
small: 118769    10.975617
small: 118770    10.975617
small: 118771    10.975617
small: 118772    10.975617
small: 118773    10.975617
small: 118774    10.975617
s

small: 119102    10.975617
small: 119103    10.975617
small: 119104    10.975617
small: 119105    10.975617
small: 119106    10.975617
small: 119107    10.975617
small: 119108    10.975617
small: 119109    10.975617
small: 119110    10.975617
small: 119111    10.975617
small: 119112    10.975617
small: 119113    10.975617
small: 119114    10.975617
small: 119115    10.975617
small: 119116    10.975617
small: 119117    10.975617
small: 119118    10.975617
small: 119119    10.975617
small: 119120    10.975617
small: 119121    10.975617
small: 119122    10.975617
small: 119123    10.975617
small: 119124    10.975617
small: 119125    10.975617
small: 119126    10.975617
small: 119127    10.975617
small: 119128    10.975617
small: 119129    10.975617
small: 119130    10.975617
small: 119131    10.975617
small: 119132    10.975617
small: 119133    10.975617
small: 119134    10.975617
small: 119135    10.975617
small: 119136    10.975617
small: 119137    10.975617
small: 119138    10.975617
s

small: 119463    10.975617
small: 119464    10.975617
small: 119465    10.975617
small: 119466    10.975617
small: 119467    10.975617
small: 119468    10.975617
small: 119469    10.975617
small: 119470    10.975617
small: 119471    10.975617
small: 119472    10.975617
small: 119473    10.975617
small: 119474    10.975617
small: 119475    10.975617
small: 119476    10.975617
small: 119477    10.975617
small: 119478    10.975617
small: 119479    10.975617
small: 119480    10.975617
small: 119481    10.975617
small: 119482    10.975617
small: 119483    10.975617
small: 119484    10.975617
small: 119485    10.975617
small: 119486    10.975617
small: 119487    10.975617
small: 119488    10.975617
small: 119489    10.975617
small: 119490    10.975617
small: 119491    10.975617
small: 119492    10.975617
small: 119493    10.975617
small: 119494    10.975617
small: 119495    10.975617
small: 119496    10.975617
small: 119497    10.975617
small: 119498    10.975617
small: 119499    10.975617
s

small: 119828    10.975617
small: 119829    10.975617
small: 119830    10.975617
small: 119831    10.975617
small: 119832    10.975617
small: 119833    10.975617
small: 119834    10.975617
small: 119835    10.975617
small: 119836    10.975617
small: 119837    10.975617
small: 119838    10.975617
small: 119839    10.975617
small: 119840    10.975617
small: 119841    10.975617
small: 119842    10.975617
small: 119843    10.975617
small: 119844    10.975617
small: 119845    10.975617
small: 119846    10.975617
small: 119847    10.975617
small: 119848    10.975617
small: 119849    10.975617
small: 119850    10.975617
small: 119851    10.975617
small: 119852    10.975617
small: 119853    10.975617
small: 119854    10.975617
small: 119855    10.975617
small: 119856    10.975617
small: 119857    10.975617
small: 119858    10.975617
small: 119859    10.975617
small: 119860    10.975617
small: 119861    10.975617
small: 119862    10.975617
small: 119863    10.975617
small: 119864    10.975617
s

small: 120184    10.975617
small: 120185    10.975617
small: 120186    10.975617
small: 120187    10.975617
small: 120188    10.975617
small: 120189    10.975617
small: 120190    10.975617
small: 120191    10.975617
small: 120192    10.975617
small: 120193    10.975617
small: 120194    10.975617
small: 120195    10.975617
small: 120196    10.975617
small: 120197    10.975617
small: 120198    10.975617
small: 120199    10.975617
small: 120200    10.975617
small: 120201    10.975617
small: 120202    10.975617
small: 120203    10.975617
small: 120204    10.975617
small: 120205    10.975617
small: 120206    10.975617
small: 120207    10.975617
small: 120208    10.975617
small: 120209    10.975617
small: 120210    10.975617
small: 120211    10.975617
small: 120212    10.975617
small: 120213    10.975617
small: 120214    10.975617
small: 120215    10.975617
small: 120216    10.975617
small: 120217    10.975617
small: 120218    10.975617
small: 120219    10.975617
small: 120220    10.975617
s